In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 130912 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.3-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.3-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.3-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
!git clone https://github.com/bgshih/aster.git

In [0]:
!sudo apt install cmake libcupti-dev
!pip install --user protobuf tqdm numpy editdistance

In [0]:
!cd aster/c_ops;sh build.sh

In [0]:
!protoc aster/protos/*.proto --python_out=.

##制作训练数据集

In [0]:
import os
import io
import copy
import random
import re
import xml.etree.ElementTree as ET
import sys
import pandas as pd

from PIL import Image, ImageFilter
import tensorflow as tf

data_dir='./'
ignore_difficult=True
crop_margin=0.2
lexicon_size = 50
random.seed(1)

class InputDataFields(object):
  image = 'image'
  original_image = 'original_image'
  key = 'key'
  source_id = 'source_id'
  filename = 'filename'
  groundtruth_text = 'groundtruth_text'
  groundtruth_keypoints = 'groundtruth_keypoints'
  lexicon = 'lexicon'


class TfExampleFields(object):
  image_encoded = 'image/encoded'
  image_format = 'image/format'  # format is reserved keyword
  filename = 'image/filename'
  channels = 'image/channels'
  colorspace = 'image/colorspace'
  height = 'image/height'
  width = 'image/width'
  source_id = 'image/source_id'
  transcript = 'image/transcript'
  lexicon = 'image/lexicon'
  keypoints = 'image/keypoints'
  
def int64_feature(value):
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


def int64_list_feature(value):
  return tf.train.Feature(int64_list=tf.train.Int64List(value=value))


def bytes_feature(value):
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))


def bytes_list_feature(value):
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=value))


def float_list_feature(value):
  return tf.train.Feature(float_list=tf.train.FloatList(value=value))


def read_examples_list(path):
  """Read list of training or validation examples.

  The file is assumed to contain a single example per line where the first
  token in the line is an identifier that allows us to find the image and
  annotation xml for that example.

  For example, the line:
  xyz 3
  would allow us to find files xyz.jpg and xyz.xml (the 3 would be ignored).

  Args:
    path: absolute path to examples list file.

  Returns:
    list of example identifiers (strings).
  """
  with tf.gfile.GFile(path) as fid:
    lines = fid.readlines()
  return [line.strip().split(' ')[0] for line in lines]


def recursive_parse_xml_to_dict(xml):
  """Recursively parses XML contents to python dict.

  We assume that `object` tags are the only ones that can appear
  multiple times at the same level of a tree.

  Args:
    xml: xml tree obtained by parsing XML file contents using lxml.etree

  Returns:
    Python dictionary holding XML contents.
  """
  if not xml:
    return {xml.tag: xml.text}
  result = {}
  for child in xml:
    child_result = recursive_parse_xml_to_dict(child)
    if child.tag != 'object':
      result[child.tag] = child_result[child.tag]
    else:
      if child.tag not in result:
        result[child.tag] = []
      result[child.tag].append(child_result[child.tag])
  return {xml.tag: result}

def _random_lexicon(lexicon_list, groundtruth_text, lexicon_size):
    lexicon = copy.deepcopy(lexicon_list)
    # del lexicon[lexicon.index(groundtruth_text.lower())]
    random.shuffle(lexicon)
    lexicon = lexicon[:(lexicon_size - 1)]
    lexicon.insert(0, groundtruth_text)
    return lexicon


def _is_difficult(word):
    assert isinstance(word, str)
    return len(word) < 3 or not re.match('^[\w]+$', word)


def create_ic03(output_path):
    writer = tf.python_io.TFRecordWriter(output_path)
    lexicon_list = []

    data = pd.read_csv('./train_id_label.csv')[38500:]
    count = 0
    for ii, row in data.iterrows():
        image_path = os.path.join('/content/train_cptn_result', row['name'])
        if not os.path.exists(image_path):
            print('%s does not exist' % image_path)
            continue
        imag = Image.open(image_path)
        image_w, image_h = imag.size
        groundtruth_text = row['label'].strip().lower()
        bbox_xmin = 0
        bbox_ymin = 0
        bbox_xmax = image_w
        bbox_ymax = image_h

        imags = []
        if '3' in row['label'].strip() and '8' not in row['label'].strip():
#       print(row['label'])
          for i in range(4):
            imags.append(imag)
        elif '5' in row['label'].strip() and '6' not in row['label'].strip() and '9' not in row['label'].strip():
          for i in range(2):
            imags.append(imag)
        else:
          imags.append(imag)
        for image in imags:
            word_crop_im = image.crop((bbox_xmin, bbox_ymin, bbox_xmax, bbox_ymax))
            im_buff = io.BytesIO()
            word_crop_im.save(im_buff, format='jpeg')
            word_crop_jpeg = im_buff.getvalue()
            crop_name = '{}:{}'.format(row['name'], ii)

            lexicon = _random_lexicon(lexicon_list, groundtruth_text, lexicon_size)

            example = tf.train.Example(features=tf.train.Features(feature={
                TfExampleFields.image_encoded: \
                    bytes_feature(word_crop_jpeg),
                TfExampleFields.image_format: \
                    bytes_feature('jpeg'.encode('utf-8')),
                TfExampleFields.filename: \
                    bytes_feature(crop_name.encode('utf-8')),
                TfExampleFields.channels: \
                    int64_feature(3),
                TfExampleFields.colorspace: \
                    bytes_feature('rgb'.encode('utf-8')),
                TfExampleFields.transcript: \
                    bytes_feature(groundtruth_text.encode('utf-8')),
                TfExampleFields.lexicon: \
                    bytes_feature(('\t'.join(lexicon)).encode('utf-8')),
            }))
            writer.write(example.SerializeToString())
            count += 1

    writer.close()
    print('{} examples created'.format(count))


if __name__ == '__main__':
    create_ic03('./ic03_val.tfrecord')

2081 examples created


###colab12小时清空一次，需要重新copy

In [0]:
# !cp ./drive/my_gpu/tinymind/ic03* ./aster/data/
!cp -r ./drive/my_gpu/tinymind/aster/core/*.py aster/core/
# !mkdir -p aster/experiments/tinymind/log;mkdir -p aster/experiments/tinymind/config
# !cp ./drive/my_gpu/tinymind/experiments/tinymind/log/* aster/experiments/tinymind/log/

In [0]:
!export LD_LIBRARY_PATH=/usr/lib64-nvidia:/usr/local/cuda-10.0/extras/CUPTI/lib64;python aster/train_1.py

### 识别图片

In [0]:
!mkdir -p ./aster/experiments/tinymind/log/
!mkdir -p ./aster/experiments/tinymind/config/

In [0]:
!cp ./drive/my_gpu/tinymind/experiments/tinymind/log/model.ckpt-110183.data-00000-of-00001 ./aster/experiments/tinymind/log/model.ckpt.data-00000-of-00001
!cp ./drive/my_gpu/tinymind/experiments/tinymind/log/model.ckpt-110183.index ./aster/experiments/tinymind/log/model.ckpt.index
!cp ./drive/my_gpu/tinymind/experiments/tinymind/log/model.ckpt-110183.meta ./aster/experiments/tinymind/log/model.ckpt.meta


In [0]:
!cp ./aster/experiments/tinymind/log/model.ckpt-110183* ./drive/my_gpu/tinymind/experiments/tinymind/log/

In [0]:
!mkdir aster_out

In [0]:
!python aster/demo_1.py